# Creating a basic chat experience with kernel arguments

In this chat scenario, as the user talks back and forth with the bot, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. 

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.3.0

Let's define the prompt.

In [2]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

Register your semantic function

In [3]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Initialize your arguments

In [4]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

Chat with the Bot

In [5]:
var userInput = "Hi, I'm looking for cocktail suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

Update the history with the output and set this as the new input value for the next request

In [6]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for cocktail suggestions
AI: Hello! I'd be happy to help. What kind of flavors do you enjoy? Sweet, sour, fruity, or something else?



Keep Chatting!

In [7]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;
    
    // Show the response
    Console.WriteLine(result);
};

In [8]:
await Chat("I would like a cocktail suggestion for breakfast. Please suggest one of them.");


User: I would like a cocktail suggestion for breakfast. Please suggest one of them.
AI: For a breakfast cocktail, you might enjoy a classic Mimosa. It's a refreshing combination of champagne and orange juice. Cheers!



In [9]:
await Chat("that sounds interesting, how do you make one?");


User: that sounds interesting, how do you make one?
AI: To make a Mimosa, you will need champagne (or sparkling wine) and orange juice. Simply pour equal parts champagne and orange juice into a glass, starting with the champagne. You can garnish with a slice of orange if you like. Enjoy your breakfast Mimosa!



In [10]:
await Chat("Write a C# code and print hello world?");


User: Write a C# code and print hello world?
AI: Sure! Here's a simple C# code that prints "Hello, World!" to the console:

```csharp
using System;

class Program
{
    static void Main()
    {
        Console.WriteLine("Hello, World!");
    }
}
```

Just copy and paste this code into a C# compiler or IDE, and you should see "Hello, World!" printed to the console when you run the program.



In [11]:
await Chat("could you list some more I could try about the topic(s) mentioned?");


User: could you list some more I could try about the topic(s) mentioned?
AI: I don't know.



After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [12]:
Console.WriteLine(history);


User: Hi, I'm looking for cocktail suggestions
AI: Hello! I'd be happy to help. What kind of flavors do you enjoy? Sweet, sour, fruity, or something else?

User: I would like a cocktail suggestion for breakfast. Please suggest one of them.
AI: For a breakfast cocktail, you might enjoy a classic Mimosa. It's a refreshing combination of champagne and orange juice. Cheers!

User: that sounds interesting, how do you make one?
AI: To make a Mimosa, you will need champagne (or sparkling wine) and orange juice. Simply pour equal parts champagne and orange juice into a glass, starting with the champagne. You can garnish with a slice of orange if you like. Enjoy your breakfast Mimosa!

User: Write a C# code and print hello world?
AI: Sure! Here's a simple C# code that prints "Hello, World!" to the console:

```csharp
using System;

class Program
{
    static void Main()
    {
        Console.WriteLine("Hello, World!");
    }
}
```

Just copy and paste this code into a C# compiler or IDE, and y